In [ ]:
from natsort import natsorted
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')

In [ ]:
path = 'Run_Testrun_PreRunStabilityTesting_FC2_4ch_5'
pattern = 'loop*/TDC*nem'
files = natsorted(Path(path).glob(pattern))

In [ ]:
filler_dict = {
    'events': [],
    'prev_event': [],
    'last_event': [],
}

In [ ]:
event_counter = 0
prev_evt = 0
for ifile in files:
    with open(ifile, 'r') as infile:
        lines = infile.readlines()

        eh_flag = False
        et_flag = False

        for iline in lines:
            flag = iline.strip().split()[0]

            if flag == 'FILLER':
                continue

            elif flag == 'CLOCK':
                filler_dict['events'].append(event_counter-prev_evt)
                filler_dict['prev_event'].append(prev_evt)
                filler_dict['last_event'].append(event_counter)
                prev_evt = event_counter

            ## Event starts
            elif flag == 'EH':
                eh_flag = True

            ## Event end
            elif flag == 'ET':
                et_flag = True

            else:
                continue

            if eh_flag & et_flag:
                event_counter += 1
                eh_flag = False
                et_flag = False

In [ ]:
window_size = 40
prescale = 2048

filler_df = pd.DataFrame(filler_dict).reset_index().astype('int32')
avg_df = filler_df['events'].rolling(window=window_size).sum().to_frame(name='avg').reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(22, 11))
hep.cms.text(loc=0, ax=ax, text="ETL ETROC SEU", fontsize=20)
ax.plot(avg_df['index']*0.025, prescale*avg_df['avg']*1e-3/(window_size*0.025*1.024**2), '.')
ax.set_xlabel('Time [s]', fontsize=25)
ax.tick_params(axis='both', labelsize=20)
ax.set_ylabel('', fontsize=25)